In [24]:
import pandas as pd
import math
import sys
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
DEFAULT_PATH = "../input/"

In [26]:
musiques = pd.read_csv(DEFAULT_PATH + "musiquecompile/dataCompile.csv")
musiques

In [27]:
df = musiques.drop(musiques.columns[:5], axis=1)
df.drop(df.columns[11:16], axis=1, inplace=True)
df.drop(["time_signature"], axis=1, inplace=True)

#Normalise data
normalized_df=(df-df.mean())/df.std()

normalized_df_and_limited = normalized_df[:100]

In [28]:
matrice = normalized_df_and_limited.to_numpy()
matrice

In [29]:
similariter = cosine_similarity(matrice)#regarder pour euclide
similariter

In [30]:
value = np.round(similariter, 4)

In [31]:
value, count = np.unique(value, return_counts=True)
value = value[:len(value)-1]
count = count[:len(count)-1]

In [32]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [33]:
moyenneMobile = list(moving_average(count, 10))
for i in range(int(((len(value) - len(moyenneMobile)) / 2))):
    moyenneMobile.insert(0, None)
    moyenneMobile.insert(0, None)
if len(value) - len(moyenneMobile) == 1:
    moyenneMobile.insert(0, None)

In [34]:
plt.plot(value, count)
plt.plot(value, moyenneMobile, "r")

In [35]:
plt.figure(1,figsize=(7.5,5)) 
sns.ecdfplot(count, x=value, complementary=True)

In [36]:
print(count, len(count), value, len(value))
count

In [37]:
similariter[similariter <= 0.5] = 0

In [38]:
g = nx.from_numpy_matrix(similariter)
g = nx.relabel_nodes(g,{i:musiques["song_name"][i] for i in range(len(normalized_df))})

In [39]:
plt.figure(1,figsize=(30,20)) 
nx.draw_networkx(g)
plt.savefig("/kaggle/working/GraphLiason.png", dpi=150, format="png")

In [40]:
from sklearn.cluster import KMeans
"""scores = []
for i in range(1, 50):
    model = KMeans(n_clusters = i, algorithm="elkan").fit(similariter)
    scores.append(model.inertia_)
plt.plot(range(1, 50), scores)"""

In [41]:
model = KMeans(n_clusters = 5)
model.fit(similariter)
data = model.fit_predict(similariter)

In [42]:
plt.figure(1,figsize=(30,20)) 
nx.draw_networkx(g, node_color=data)
plt.savefig("/kaggle/working/GraphLiasonWithGroups.png", dpi=150, format="png")

In [43]:
#Recuperation des voisins,
g.adj["She's Alright"]

In [44]:
#Triage des voisin pour prendre celui qui a le plu fort weight
def getMusiqueSimilar(musiqueTitle):
    #Init
    titleMusique = []
    weight = []
    #Recuperation et création des listes
    for i in g.adj[musiqueTitle].items():
        titleMusique.append(i[0])
        weight.append(i[1]['weight'])

    #Creation de DataFrame
    #Creation d'un DataFrame pour pouvoir tier par leurs poids
    data = {"title": titleMusique, "weight": weight}
    sameMusique = pd.DataFrame(data=data)
    sameMusique.set_index("title", inplace=True)
    sameMusique.sort_values("weight", inplace=True, ascending=False)
    return sameMusique

In [45]:
#Triage des voisin pour prendre celui qui a le plu fort weight
def getMusiqueMostSimilar(listMusique):
    #Init
    listMusiqueStockage = []
    musiqueData = {}
    
    #Recuperation et création des listes
    for musiqueTitle in listMusique:
        for i in g.adj[musiqueTitle].items():
            musiqueData[i[0]] = i[1]['weight']
        listMusiqueStockage.append(musiqueData)
        musiqueData = {}

    resultIntersectionSet = set([i[0] for i in list(listMusiqueStockage[0].items())])
    for page in listMusiqueStockage:
        resultIntersectionSet = resultIntersectionSet & set([i[0] for i in list(page.items())])#Recupération des nom des musiques
    
    nameMusique_Weight = {}
    for nameMusique in resultIntersectionSet:
        for listMusiqueIndex in listMusiqueStockage:
            if nameMusique in listMusiqueIndex:
                if nameMusique in nameMusique_Weight:
                    nameMusique_Weight[nameMusique] = (listMusiqueIndex[nameMusique] + nameMusique_Weight[nameMusique]) / 2
                else:
                    nameMusique_Weight[nameMusique] = listMusiqueIndex[nameMusique]
            
    data = {"title": [i for i in nameMusique_Weight.keys()], "weight": [i for i in nameMusique_Weight.values()]}
    return pd.DataFrame(data=data).set_index("title").sort_values("weight", ascending=False)#list(resultIntersectionDic)

In [46]:
all_normalized_df = normalized_df
matrice = all_normalized_df.to_numpy()
similariter = cosine_similarity(matrice)#regarder pour euclide
similariter[similariter <= 0.5] = 0
g = nx.from_numpy_matrix(similariter)
g = nx.relabel_nodes(g,{i:musiques["song_name"][i]+'|'+musiques["artiste_name"][i] for i in range(len(normalized_df))})

In [47]:
#getMusiqueSimilar("Three")

In [48]:
def getGraphForMusique(title):
    listAllNode = list(g.adj[title])[:100]
    sameMusique = getMusiqueSimilar(title)

    graphMusiqueChoose = nx.Graph()
    graphMusiqueChoose.add_nodes_from(listAllNode)
    for i in listAllNode:
        graphMusiqueChoose.add_edge(title, i, weight=math.exp(sameMusique.loc[i]*4)/10)
    
    weights = [graphMusiqueChoose[u][v]['weight'] for u,v in graphMusiqueChoose.edges()]
    plt.figure(1,figsize=(30,20)) 
    nx.draw_networkx(graphMusiqueChoose, width=[float(i) for i in weights])

In [49]:
getGraphForMusique("Three|The Show is the Rainbow")
plt.savefig("/kaggle/working/allSimilarMusique.png", dpi=150, format="png")

In [50]:
getMusiqueMostSimilar(list(["Three|The Show is the Rainbow", "(Face)|Septembre"]))

In [51]:
musiquesIdSongName = musiques.set_index("song_name")

In [52]:
musiquesIdSongName.loc["Silent Night"].iloc[0]

In [53]:
def getMusiqueData(listAllMusiqueName):
    allDataMusique = []
    for i in listAllMusiqueName:
        musiqueInfoReachTab = i.split("|")
        data = musiquesIdSongName.loc[musiqueInfoReachTab[0]]
        if (len(data.shape) > 1):
            for j in range(data.shape[0]):
                if musiqueInfoReachTab[1] == data["artiste_name"].iloc[j]:
                    allDataMusique.append(data.iloc[j])
        else:
            allDataMusique.append(data)
    return allDataMusique

In [54]:
getMusiqueData(list(["Three|The Show is the Rainbow"]))

In [55]:
def makeDataReturn(listMusique):
    urlMusique = []
    for i in listMusique:
        urlMusique.append(i["track_href"])
    # print(urlMusique)
    data = {"title": [i for i in urlMusique]}
    return pd.DataFrame(data=data)